# Template of a SMI jupyter notebook

In [ ]:
# These packages should all be installed if the procedure was followed
%pylab nbagg

import matplotlib.pyplot as plt

from smi_analysis import SMI_beamline
import numpy as np
import fabio
import os

## Experimental configuration

In [ ]:
geometry = 'Reflection'
energy = 2.45
wav = 1E-10 * (12.398/energy)
bs_kind = None
det_ini_angle = np.deg2rad(0)
det_angle_step = np.deg2rad(6.5)
incident_angle = np.deg2rad(0)

#WAXS
detector_waxs = 'Pilatus900kw'
sdd_waxs = 277
center_waxs = [95., 1256]
bs_pos_waxs = [[97, 1070], [0, 0], [0, 0]]

SMI_waxs = SMI_beamline.SMI_geometry(geometry = geometry,
                                     detector = detector_waxs,
                                     sdd = sdd_waxs,
                                     wav = wav,
                                     alphai = incident_angle,
                                     center = center_waxs,
                                     bs_pos = bs_pos_waxs,
                                     det_ini_angle = det_ini_angle,
                                     det_angle_step = det_angle_step,
                                     bs_kind = bs_kind)

## Locate the files on your computer

In [ ]:
CYCLE='2024_1'
proposal_id =  '313064_Davis'

path = '/nsls2/data/smi/legacy/results/data/%s/%s/900KW/'%(CYCLE, proposal_id)

filelist=os.listdir(path)
for file in filelist:
    if 'test' not in file:
        print(file)
print(len(filelist))

In [ ]:
#waxs
path = '/nsls2/data/smi/legacy/results/data/%s/%s/900KW/'%(CYCLE, proposal_id)
ResDir = '/nsls2/data/smi/legacy/results/analysis/%s/%s/reduced_giwaxs/'%(CYCLE, proposal_id)


sam = []
for file in os.listdir(path):
     if 'wa20' in file and 'test' not in file and 'num' not in file:
        idx1 = file.find('wa20')
        if file[:idx1] not in sam:
            sam = sam + [file[:idx1]]

print(sam)

all_dat = [[]] * len(sam)
all_da = [[]] * len(sam)
    
for j, sa in enumerate(sam): 
    for file in sorted(os.listdir(path)):
        if sa in file and 'tif' in file and 'num' not in file:
            all_dat[j] = all_dat[j] + [file]
            
print(all_dat[0])
    
for i, all_d in enumerate(all_dat):
    for wa in ['wa0', 'wa20']:
        for dat in all_d:
            if wa in dat:
                all_da[i] = all_da[i] + [dat]

print(all_da[0])

In [ ]:
len(all_da)/2

In [ ]:
all_da

## Data stitching and reduction (all this part is done using SMI_beamline package)

In [ ]:
all_da[50]

In [ ]:
#waxs
flag_save = False

for j, dat in enumerate([all_da[50]]):
        idx = dat[0].find('eV')
        energy = 0.001 * float(dat[0][idx-7:idx])
        wav = 1E-10 * (12.398/energy)
           
        waxs_angle = []
        for da in dat:  
            idx2 = da.find('wa')
            idx3 = da.find('_bpm')
            waxs_angle = waxs_angle + [np.deg2rad(float(da[idx2+2:idx3]))]
            
        alphai=np.deg2rad(0.8)
            
        SMI_waxs = SMI_beamline.SMI_geometry(geometry = geometry,
                                             detector = 'Pilatus900kw',
                                             sdd = sdd_waxs,
                                             wav = wav,
                                             alphai = alphai,
                                             center = center_waxs,
                                             bs_pos = bs_pos_waxs,
                                             det_angles = waxs_angle,
                                             bs_kind = None)

        SMI_waxs.open_data(path, dat)
        
#         plt.figure()
#         for i, (img, mask) in enumerate(zip(SMI_waxs.imgs, SMI_waxs.masks)):
#             SMI_waxs.imgs[i] = (SMI_waxs.imgs[i])
#             plt.subplot(1, len(SMI_waxs.imgs), i+1)
#             plt.imshow(SMI_waxs.imgs[i]*np.logical_not(mask), vmin = 0, vmax = np.percentile(SMI_waxs.imgs[0], 99))
#         plt.show()

        SMI_waxs.stitching_data(interp_factor=3)

        plt.figure()
        plt.imshow(SMI_waxs.img_st,
                   extent=[SMI_waxs.qp[0], SMI_waxs.qp[-1], SMI_waxs.qz[0], SMI_waxs.qz[-1]], 
                   vmin=0,
                   vmax=np.percentile(SMI_waxs.img_st, 99))    
        plt.title('waxs_%s'%sam[j])
        plt.xlabel('$q_{par}(A^{-1})$')
        plt.ylabel('$q_{ver}(A^{-1})$')
        plt.colorbar()
        plt.show()

        
#         SMI_waxs.radial_averaging(radial_range = [0, 1.3], azimuth_range=[0, 90], npt = 2000)
#         q0, I0 = SMI_waxs.q_rad, SMI_waxs.I_rad

#         SMI_waxs.radial_averaging(radial_range = [0, 1.3], azimuth_range=[0, 30], npt = 2000)
#         q1, I1 = SMI_waxs.q_rad, SMI_waxs.I_rad

#         SMI_waxs.radial_averaging(radial_range = [0, 1.3], azimuth_range=[60, 90], npt=2000)
#         q2, I2 = SMI_waxs.q_rad, SMI_waxs.I_rad

#         plt.figure()
#         plt.plot(q0, I0, linewidth = 3, color = 'k')
#         plt.plot(q1, I1, linewidth = 3, color = 'g')
#         plt.plot(q2, I2, linewidth = 3, color = 'r')
# #         plt.xlim(0.004, 0.9)
# #         plt.ylim(2E5, 2E9)

#         plt.xscale('log')
#         plt.yscale('log')
#         plt.xlabel('q (A-1)', fontsize=12)
#         plt.ylabel('Intensity (a. u)', fontsize=12)

#         plt.show()

print('Done')

In [ ]:
#     if 'q_map_waxs_%s_pos%s.png'%(sam[j], sam1[j]) not in os.listdir(ResDir):

In [ ]:
#waxs
flag_save = False

for j, dat in enumerate(all_da):
    print(dat[0])
    if len(dat)==2 and 'Radint_ver_waxs_%s.txt'%(sam[j]) not in os.listdir(ResDir):
        idx = dat[0].find('eV')
        energy = 0.001 * float(dat[0][idx-7:idx])
        wav = 1E-10 * (12.398/energy)
           
        waxs_angle = []
        for da in dat:  
            idx2 = da.find('wa')
            idx3 = da.find('_bpm')
            waxs_angle = waxs_angle + [np.deg2rad(float(da[idx2+2:idx3]))]
            
        alphai=np.deg2rad(0.8)
            
        SMI_waxs = SMI_beamline.SMI_geometry(geometry = geometry,
                                             detector = 'Pilatus900kw',
                                             sdd = sdd_waxs,
                                             wav = wav,
                                             alphai = alphai,
                                             center = center_waxs,
                                             bs_pos = bs_pos_waxs,
                                             det_angles = waxs_angle,
                                             bs_kind = None)

        SMI_waxs.open_data(path, dat)

        SMI_waxs.stitching_data(interp_factor=3)
        

#         plt.figure()
#         plt.imshow(SMI_waxs.img_st,
#                    extent=[SMI_waxs.qp[0], SMI_waxs.qp[-1], SMI_waxs.qz[0], SMI_waxs.qz[-1]], 
#                    vmin=0,
#                    vmax=np.percentile(SMI_waxs.img_st, 97))    
#         plt.title('waxs_%s'%sam[j])
#         plt.xlabel('$q_{par}(A^{-1})$')
#         plt.ylabel('$q_{ver}(A^{-1})$')
#         plt.colorbar()
#         plt.show()
        
        SMI_waxs.radial_averaging(radial_range = [0, 1.3], azimuth_range=[0, 90], npt = 2000)
        q0, I0 = SMI_waxs.q_rad, SMI_waxs.I_rad

        SMI_waxs.radial_averaging(radial_range = [0, 1.3], azimuth_range=[0, 30], npt = 2000)
        q1, I1 = SMI_waxs.q_rad, SMI_waxs.I_rad

        SMI_waxs.radial_averaging(radial_range = [0, 1.3], azimuth_range=[60, 90], npt=2000)
        q2, I2 = SMI_waxs.q_rad, SMI_waxs.I_rad

        if flag_save:
#             #Save the plot as a png
#             plt.savefig(os.path.join(ResDir, 'q_map_waxs_%s_pos%s.png'%(sam[j], sam1[j])))

#             #Save the qx and qy as an 1D array
#             np.savetxt(os.path.join(ResDir, 'qpar_waxs_%s_pos%s.txt'%(sam[j], sam1[j])), np.linspace(SMI_waxs.qp[0], SMI_waxs.qp[1], np.shape(SMI_waxs.img_st)[1]))
#             np.savetxt(os.path.join(ResDir, 'qver_waxs_%s_pos%s.txt'%(sam[j], sam1[j])), np.linspace(SMI_waxs.qz[0], SMI_waxs.qz[1], np.shape(SMI_waxs.img_st)[0])) 

#             #Save the stitched image as a tiff
#             fabio.tifimage.TifImage(np.nan_to_num(SMI_waxs.img_st).astype('int')).write(os.path.join(ResDir, 'q_map_waxs_%s_pos%s.tiff'%(sam[j], sam1[j]))) 

            #Save the sum image as a tiff
            np.savetxt(os.path.join(ResDir, 'Radint_tot_waxs_%s.txt'%(sam[j])), np.vstack((q0, I0)).T)
            np.savetxt(os.path.join(ResDir, 'Radint_ver_waxs_%s.txt'%(sam[j])), np.vstack((q1, I1)).T)
            np.savetxt(os.path.join(ResDir, 'Radint_hor_waxs_%s.txt'%(sam[j])), np.vstack((q2, I2)).T)

#         plt.close() 
print('Done')

In [ ]:
float(dat[0][idx-7:idx])


In [ ]:
dat[0]